#BASIC FEATURE EXTRACTION

In [131]:
df["num_words"] = df["question_text"].apply(lambda x: len(str(x).split()))


## Number of unique words in the text ##
df["num_unique_words"] = df["question_text"].apply(lambda x: len(set(str(x).split())))


## Number of characters in the text ##
df["char_count"] = df["question_text"].apply(lambda x: len(str(x)))


## Number of stopwords in the text ##
df["stopword_count"] = df["question_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))


## Number of punctuations in the text ##
df["punctuations_count"] =df['question_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )


## Number of title case words in the text ##
df["num_words_upper"] = df["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the text ##
df["num_words_title"] = df["question_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

## Average length of the words in the text ##
df["mean_word_len"] = df["question_text"].apply(lambda x:avg_word(x))


In [ ]:
#https://pypi.org/project/textstat/
#All the theory is well explained on the above link and wikipedia

In [13]:
import textstat
df['flesch_reading_ease']=df['question_text'].apply(lambda x:textstat.flesch_reading_ease(x))
df['flesch_kincaid_grade']=df['question_text'].apply(lambda x:textstat.flesch_kincaid_grade(x))
df['textstat.coleman_liau_index']=df['question_text'].apply(lambda x:textstat.coleman_liau_index(x))
df['textstat.automated_readability_index']=df['question_text'].apply(lambda x:textstat.automated_readability_index(x))
df['textstat.dale_chall_readability_score']=df['question_text'].apply(lambda x:textstat.dale_chall_readability_score(x))
df['textstat.smog_index']=df['question_text'].apply(lambda x:textstat.smog_index(x))
df['gunning_fog']=df['question_text'].apply(lambda x:textstat.gunning_fog(x))
#df['text_standard']=df['question_text'].apply(lambda x:textstat.text_standard(x))
df['linsear_write_formula']=df['question_text'].apply(lambda x:textstat.linsear_write_formula(x))
df['difficult_words']=df['question_text'].apply(lambda x:textstat.difficult_words(x))                                                   

In [19]:
#lets see their usefulness

In [65]:
pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def pos_check(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt

df['noun_count'] = df['question_text'].apply(lambda x: pos_check(x, 'noun'))
df['verb_count'] = df['question_text'].apply(lambda x: pos_check(x, 'verb'))
df['adj_count'] = df['question_text'].apply(lambda x: pos_check(x, 'adj'))
df['adv_count'] = df['question_text'].apply(lambda x: pos_check(x, 'adv'))
df['pron_count'] = df['question_text'].apply(lambda x: pos_check(x, 'pron'))

#TOPIC MODELLING

In [147]:
def add_features(df):
    df["num_words"] = df["question_text"].apply(lambda x: len(str(x).split()))
## Number of unique words in the text ##
    df["num_unique_words"] = df["question_text"].apply(lambda x: len(set(str(x).split())))
## Number of characters in the text ##
    df["char_count"] = df["question_text"].apply(lambda x: len(str(x)))
## Number of stopwords in the text ##
    df["stopword_count"] = df["question_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
## Number of punctuations in the text ##
    df["punctuations_count"] =df['question_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
## Number of title case words in the text ##
    df["num_words_upper"] = df["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
## Number of title case words in the text ##
    df["num_words_title"] = df["question_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
## Average length of the words in the text ##
    df["mean_word_len"] = df["question_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

    df['text_polarity'] = df['question_text'].apply(lambda x: getpolarity(x))
    df['text_subjectivity'] = df['question_text'].apply(lambda x: getsubjectivity(x))  
    df["lower_question_text"] = df["question_text"].apply(lambda x: x.lower())
    df['num_exclamation_marks'] = df['question_text'].apply(lambda comment: comment.count('!'))
    df['num_question_marks'] = df['question_text'].apply(lambda comment: comment.count('?'))
    df['num_symbols'] = df['question_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
    df['num_smilies'] = df['question_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
    df['num_sad'] = df['question_text'].apply(lambda comment: sum(comment.count(w) for w in (':-<', ':()', ';-()', ';(')))
    
    # Number of F word
    df["nb_fk"] = df["question_text"].apply(lambda x: count_regexp_occ(r"[Ff]\S{2}[Kk]", x))
    # Number of S word
    df["nb_sk"] = df["question_text"].apply(lambda x: count_regexp_occ(r"[Ss]\S{2}[Kk]", x))
    # Number of D words
    df["nb_dk"] = df["question_text"].apply(lambda x: count_regexp_occ(r"[dD]ick", x))
    # Number of occurence of You, insulting someone usually needs someone called : you
    df["nb_you"] = df["question_text"].apply(lambda x: count_regexp_occ(r"\W[Yy]ou\W", x))
    # Just to check you really refered to my mother ;-)
    df["nb_mother"] = df["question_text"].apply(lambda x: count_regexp_occ(r"\W[Mm]other\W", x))
    # Just checking for toxic 19th century vocabulary
    df["nb_ng"] = df["question_text"].apply(lambda x: count_regexp_occ(r"\W[Nn]igger\W", x))
    df['sum_expletive'] = df["nb_fk"] + df["nb_sk"] + df["nb_dk"] + df["nb_ng"]
    
    df['people_person'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in ('people', 'person')))
    df['trump'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in ('trump','donald','president')))
    df['women'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in ('woman','women', 'girl', 'girls','womens')))
    df['men'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in ('man','men','mens')))
    df['race'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in ('white','black','whites','blacks')))
    df['china'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in ('china','chinese')))
    df['india'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in ('india','indian','indians')))
    df['country'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in ('country','countries')))
    df['quora'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in (['quora'])))
    df['america'] = df["lower_question_text"].apply(lambda comment: sum(comment.count(w) for w in ('america','american','americans')))
    df['sum_feat'] = df["nb_you"]+df["nb_mother"]+df['sum_expletive']+df['people_person']+df['trump']+df['women']+df['men']+df['race']+df['china']+df['india']+df['country']+df['quora']+df['america']
    df['why'] = df["lower_question_text"].apply(lambda comment: fword(comment,'why'))
    df['howwhat'] = df["lower_question_text"].apply(lambda comment: fword(comment,'how')+ fword(comment,'what'))
    df['doare'] = df["lower_question_text"].apply(lambda comment: fword(comment,'do')+ fword(comment,'are'))
    return df

In [148]:
def getpolarity(s):
    sent = TextBlob(s).sentiment
    return sent.polarity
def  getsubjectivity(s):
    sent = TextBlob(s).sentiment
    return sent.subjectivity

def count_regexp_occ(regexp="", text=None):
    """ Simple way to get the number of occurence of a regex"""
    return len(re.findall(regexp, text))

def fword(x,word):
    if (x.split(" ")[0].lower() ==word):
        return 1
    else: return 0

In [149]:
df=add_features(df)

In [150]:
import nltk

In [151]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' ')
    return x
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 
                'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 
                'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 
                'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do',
                'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 
                'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum',
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 
                'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 
                'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

stopword_list = nltk.corpus.stopwords.words('english')
def remove_stopwords(text, is_lower_case=True):
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

#wordnet_lemmatizer = WordNetLemmatizer()
def lemma_text(text):
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)


In [152]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                       "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not",
                       "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", 
                       "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                       "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", 
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have"
                       ,"i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                       "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                       "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                       "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not",
                       "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                       "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
                       "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have",
                       "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                       "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is",
                       "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                       "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                       "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
                       "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
                       "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", 
                       "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
                       "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have"
                       , "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                       "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                       "y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


In [153]:
#fxn for preprocessing
def clean_sentence(x):
    x = x.lower()
    x = clean_text(x)
    x = replace_typical_misspell(x)
    x = remove_stopwords(x)
    x = replace_contractions(x)
    #x = lemma_text(x)
    x = x.replace("'","")
    return x

In [154]:
#df['question_te']=df['question_text'].apply(lambda x: replace_contractions(x))

In [155]:
df['cleaned_text'] = df['question_text'].apply(lambda x : clean_sentence(x))

In [156]:
df['cleaned_text'][0:5]

0        quebec nationalists see province nation 1960s
1        adopted dog would encourage people adopt shop
2    velocity affect time velocity affect space geo...
3         otto von guericke used magdeburg hemispheres
4    convert montra helicon mountain bike changing ...
Name: cleaned_text, dtype: object

In [157]:
#best threshold fxn
def bestThresshold(y_train,train_preds):
    tmp = [0,0,0] # idx, cur, max
    delta = 0
    for tmp[0] in tqdm(np.arange(0.1, 0.501, 0.01)):
        tmp[1] = f1_score(y_train, np.array(train_preds)>tmp[0])
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))
    return tmp[2]

In [158]:
y=df["target"]

In [159]:
def model_train_cv(x_train,y_train,nfold,model_obj):
    splits = list(StratifiedKFold(n_splits=nfold, shuffle=True, random_state=1028).split(x_train, y_train))
    x_train = x_train
    y_train = np.array(y_train)
    # matrix for the out-of-fold predictions
    train_oof_preds = np.zeros((x_train.shape[0]))
    for i, (train_idx, valid_idx) in enumerate(splits):

        x_train_fold = x_train[train_idx.astype(int)]
        y_train_fold = y_train[train_idx.astype(int)]
        x_val_fold = x_train[valid_idx.astype(int)]
        y_val_fold = y_train[valid_idx.astype(int)]

        clf = copy.deepcopy(model_obj)
        clf.fit(x_train_fold, y_train_fold)
        valid_preds_fold = clf.predict_proba(x_val_fold)[:,1]

        # storing OOF predictions
        train_oof_preds[valid_idx] = valid_preds_fold
    return train_oof_preds

In [160]:
#Applying algorithms now:

In [171]:
train=df.drop(['question_text','cleaned_text','target','qid','lower_question_text'],axis=1)
#valid=X_valid.drop(['question_text','cleaned_text','target','qid','lower_question_text'],axis=1)

In [173]:
print(train.shape)


(1306122, 36)


In [174]:
cnt_vectorizer = CountVectorizer(dtype=np.float32,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),min_df=3)

# Fitting count vectorizer to both training and test sets (semi-supervised learning)
cnt_vectorizer.fit(df['cleaned_text'])
Xtrain =  cnt_vectorizer.transform(df['cleaned_text']) 
#xtest_cntv = cnt_vectorizer.transform(test_df.cleaned_text.values)
#Xvalid= cnt_vectorizer.transform(X_valid['cleaned_text']) 

In [175]:
from scipy.sparse import coo_matrix, hstack
Xtrain=hstack([Xtrain,train])
#Xvalid=hstack([Xvalid,valid])

In [176]:
Xtrain.shape

(1306122, 549302)

In [177]:
x_train=Xtrain.tocsr()


In [179]:
train_oof_preds = model_train_cv(x_train,y,5,LogisticRegression(C=1.0))

print ("F1 Score: %0.3f " % bestThresshold(y,train_oof_preds))


best threshold is 0.2200 with F1 score: 0.6234
F1 Score: 0.623 


In [180]:
def lgb_model_train_cv(x_train,y_train,nfold,lgb):
    splits = list(StratifiedKFold(n_splits=nfold, shuffle=True, random_state=SEED).split(x_train, y_train))
    x_train = x_train
    y_train = np.array(y_train)
    # matrix for the out-of-fold predictions
    train_oof_preds = np.zeros((x_train.shape[0]))
    for i, (train_idx, valid_idx) in enumerate(splits):
        x_train_fold = x_train[train_idx.astype(int)]
        y_train_fold = y_train[train_idx.astype(int)]
        x_val_fold = x_train[valid_idx.astype(int)]
        y_val_fold = y_train[valid_idx.astype(int)]
        d_train = lgb.Dataset(x_train_fold, label=y_train_fold)
        d_val = lgb.Dataset(x_val_fold, label=y_val_fold)
        params = {}
        params['learning_rate'] = 0.01
        params['boosting_type'] = 'gbdt'
        params['objective'] = 'binary'
        params['metric'] = 'binary_logloss'
        params['sub_feature'] = 0.5
        params['num_leaves'] = 10
        params['min_data'] = 50
        params['max_depth'] = 10
        
        clf = lgb.train(params, d_train, num_boost_round = 100,valid_sets=(d_val), early_stopping_rounds=10,verbose_eval=10)
        valid_preds_fold = clf.predict(x_val_fold)
        # storing OOF predictions
        train_oof_preds[valid_idx] = valid_preds_fold
    return train_oof_preds

In [181]:
#LGBM

In [182]:
import lightgbm as lgb
SEED=1028
train_oof_preds = lgb_model_train_cv(x_train,y,5,lgb)


Training until validation scores don't improve for 10 rounds.
[10]	valid_0's binary_logloss: 0.221304
[20]	valid_0's binary_logloss: 0.213164
[30]	valid_0's binary_logloss: 0.206825
[40]	valid_0's binary_logloss: 0.201629
[50]	valid_0's binary_logloss: 0.197269
[60]	valid_0's binary_logloss: 0.193689
[70]	valid_0's binary_logloss: 0.190401
[80]	valid_0's binary_logloss: 0.187697
[90]	valid_0's binary_logloss: 0.185149
[100]	valid_0's binary_logloss: 0.182886
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.182886
Training until validation scores don't improve for 10 rounds.
[10]	valid_0's binary_logloss: 0.221167
[20]	valid_0's binary_logloss: 0.212873
[30]	valid_0's binary_logloss: 0.206807
[40]	valid_0's binary_logloss: 0.201952
[50]	valid_0's binary_logloss: 0.197481
[60]	valid_0's binary_logloss: 0.193833
[70]	valid_0's binary_logloss: 0.190556
[80]	valid_0's binary_logloss: 0.187774
[90]	valid_0's binary_logloss: 0.185273
[100]	valid_0's binary_log

In [184]:
print ("F1 Score: %0.3f " % bestThresshold(y,train_oof_preds))



best threshold is 0.1300 with F1 score: 0.4618
F1 Score: 0.462 


In [185]:
#TFIDF

In [186]:
tfv = TfidfVectorizer(dtype=np.float32, min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(df['cleaned_text'].values) )
xtrain =  tfv.transform(df['cleaned_text'].values) 
#xtest_tfv = tfv.transform(test_df.cleaned_text.values)
#xvalid=tfv.transform(X_valid['cleaned_text'].values)


In [187]:
xtrain.shape

(1306122, 482781)

In [188]:
import scipy
from scipy.sparse import coo_matrix, hstack
Xtrain=scipy.sparse.hstack((xtrain,train))
#Xvalid=scipy.sparse.hstack((xvalid,valid))

In [191]:
Xtrain=Xtrain.tocsr()

In [192]:
train_oof_preds = model_train_cv(Xtrain,y,5,LogisticRegression(C=1.0))

print ("F1 Score: %0.3f " % bestThresshold(y,train_oof_preds))


best threshold is 0.2400 with F1 score: 0.6240
F1 Score: 0.624 


In [193]:
train_oof_preds = lgb_model_train_cv(Xtrain,y,5,lgb)
print ("F1 Score: %0.3f " % bestThresshold(y,train_oof_preds))

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's binary_logloss: 0.221327
[20]	valid_0's binary_logloss: 0.212847
[30]	valid_0's binary_logloss: 0.206666
[40]	valid_0's binary_logloss: 0.201762
[50]	valid_0's binary_logloss: 0.197447
[60]	valid_0's binary_logloss: 0.193861
[70]	valid_0's binary_logloss: 0.190467
[80]	valid_0's binary_logloss: 0.187586
[90]	valid_0's binary_logloss: 0.185069
[100]	valid_0's binary_logloss: 0.182841
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.182841
Training until validation scores don't improve for 10 rounds.
[10]	valid_0's binary_logloss: 0.221881
[20]	valid_0's binary_logloss: 0.214171
[30]	valid_0's binary_logloss: 0.20788
[40]	valid_0's binary_logloss: 0.202406
[50]	valid_0's binary_logloss: 0.198059
[60]	valid_0's binary_logloss: 0.194167
[70]	valid_0's binary_logloss: 0.190854
[80]	valid_0's binary_logloss: 0.187888
[90]	valid_0's binary_logloss: 0.185292
[100]	valid_0's binary_logl


best threshold is 0.1300 with F1 score: 0.4615
F1 Score: 0.462 


In [ ]:
#hashing 

In [46]:
import lightgbm as lgb

In [194]:
hv = HashingVectorizer(dtype=np.float32,
            strip_accents='unicode', analyzer='word',
            ngram_range=(1, 3),n_features=2**10,non_negative=True)
# Fitting Hash Vectorizer to both training and test sets (semi-supervised learning)
hv.fit(list(df.cleaned_text.values))
xtrain =  hv.transform(df.cleaned_text.values) 
#xtest_hv = hv.transform(test_df.cleaned_text.values)
#y_train = train_df.target.values

In [195]:
import scipy
from scipy.sparse import coo_matrix, hstack
Xtrain=scipy.sparse.hstack((xtrain,train))
#Xvalid=scipy.sparse.hstack((xvalid,valid))

In [197]:
Xtrain=Xtrain.tocsr()

In [198]:
train_oof_preds = model_train_cv(Xtrain,y,5,LogisticRegression(C=1.0))

print ("F1 Score: %0.3f " % bestThresshold(y,train_oof_preds))


best threshold is 0.2000 with F1 score: 0.4673
F1 Score: 0.467 


In [199]:
train_oof_preds = lgb_model_train_cv(Xtrain,y,5,lgb)
print ("F1 Score: %0.3f " % bestThresshold(y,train_oof_preds))

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's binary_logloss: 0.222411
[20]	valid_0's binary_logloss: 0.215407
[30]	valid_0's binary_logloss: 0.209223
[40]	valid_0's binary_logloss: 0.204264
[50]	valid_0's binary_logloss: 0.200228
[60]	valid_0's binary_logloss: 0.196899
[70]	valid_0's binary_logloss: 0.193981
[80]	valid_0's binary_logloss: 0.191363
[90]	valid_0's binary_logloss: 0.189048
[100]	valid_0's binary_logloss: 0.186829
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.186829
Training until validation scores don't improve for 10 rounds.
[10]	valid_0's binary_logloss: 0.22294
[20]	valid_0's binary_logloss: 0.215522
[30]	valid_0's binary_logloss: 0.209465
[40]	valid_0's binary_logloss: 0.204439
[50]	valid_0's binary_logloss: 0.200119
[60]	valid_0's binary_logloss: 0.196649
[70]	valid_0's binary_logloss: 0.193636
[80]	valid_0's binary_logloss: 0.191116
[90]	valid_0's binary_logloss: 0.188788
[100]	valid_0's binary_logl


best threshold is 0.1300 with F1 score: 0.4298
F1 Score: 0.430 
